In [1]:
from keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd
from keras import optimizers, regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Flatten, GRU
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils, to_categorical
from keras.layers import Embedding
from sklearn.model_selection import train_test_split
import pickle
import re
import operator
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

Using TensorFlow backend.
/Users/aaronfrederick/Desktop/venv/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [2]:
corpus = pickle.load(open('corpus.pkl', 'rb'))
cleaned = [re.sub('[^a-zA-Z ]', '', song) for song in corpus]

cleaned = list(set(cleaned))
tokens = [song.split() for song in cleaned]

words = []
for song in cleaned:
    for word in song.split():
        words.append(word)

lines = []
for i in range(len(words)-7):
    lines.append(words[i:i+7])

In [3]:
# lines_unique = []
# for line in tqdm(lines):
#     if line not in lines_unique:
#         lines_unique.append(line)

# len(lines_unique)

In [4]:
# pickle.dump(lines_unique, open('unique_lines.pkl', 'wb'))

In [5]:
lines_unique = pickle.load(open('unique_lines.pkl', 'rb'))

In [6]:
#Turn words to ints
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines_unique)
sequences = tokenizer.texts_to_sequences(lines_unique)
vocab_size = len(tokenizer.word_index) + 1

sequences = np.array(sequences)

In [7]:
sequences.shape

(143265, 7)

In [8]:
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [9]:
#Create model *DONT USE DROPOUT - DEPENDENCY ERROR*
model = Sequential()
model.add(Embedding(vocab_size, output_dim=50, input_length=seq_length))
model.add(GRU(256, return_sequences=True))
model.add(GRU(256))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [10]:
sgd = optimizers.SGD(lr=0.1, decay=0, momentum=0.05, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 6, 50)             270250    
_________________________________________________________________
gru_1 (GRU)                  (None, 6, 256)            235776    
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               393984    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               25700     
_________________________________________________________________
dense_2 (Dense)              (None, 5405)              545905    
Total params: 1,471,615
Trainable params: 1,471,615
Non-trainable params: 0
_________________________________________________________________


In [11]:
filepath = "models/weights.{epoch:02d}-{loss:.2f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor="loss", verbose=1, save_best_only=False, mode="min", period = 5)
callbacks_list = [checkpoint]

model.fit(X, y, batch_size=256, epochs=150, callbacks=callbacks_list)

Epoch 1/150
143265/143265 [==============================] - 94s 659us/step - loss: 6.8062 - acc: 0.0435
Epoch 2/150
143265/143265 [==============================] - 94s 658us/step - loss: 6.0311 - acc: 0.0424
Epoch 3/150
143265/143265 [==============================] - 94s 657us/step - loss: 6.0097 - acc: 0.0438
Epoch 4/150
143265/143265 [==============================] - 104s 728us/step - loss: 6.0004 - acc: 0.0443
Epoch 5/150
143265/143265 [==============================] - 137s 953us/step - loss: 5.9917 - acc: 0.0450

Epoch 00005: saving model to models/weights.05-5.99-bigger.hdf5
Epoch 6/150
143265/143265 [==============================] - 208s 1ms/step - loss: 5.9842 - acc: 0.0448
Epoch 7/150
143265/143265 [==============================] - 192s 1ms/step - loss: 5.9785 - acc: 0.0449
Epoch 8/150
143265/143265 [==============================] - 172s 1ms/step - loss: 5.9708 - acc: 0.0461
Epoch 9/150
143265/143265 [==============================] - 143s 1ms/step - loss: 5.9600 - acc:

Epoch 70/150
143265/143265 [==============================] - 102s 712us/step - loss: 4.2872 - acc: 0.1812

Epoch 00070: saving model to models/weights.70-4.29-bigger.hdf5
Epoch 71/150
143265/143265 [==============================] - 97s 679us/step - loss: 4.2679 - acc: 0.1836
Epoch 72/150
143265/143265 [==============================] - 98s 682us/step - loss: 4.2460 - acc: 0.1860
Epoch 73/150
143265/143265 [==============================] - 95s 664us/step - loss: 4.2246 - acc: 0.1878
Epoch 74/150
143265/143265 [==============================] - 107s 748us/step - loss: 4.2058 - acc: 0.1894
Epoch 75/150
143265/143265 [==============================] - 108s 753us/step - loss: 4.1844 - acc: 0.1926

Epoch 00075: saving model to models/weights.75-4.18-bigger.hdf5
Epoch 76/150
143265/143265 [==============================] - 105s 731us/step - loss: 4.1648 - acc: 0.1938
Epoch 77/150
143265/143265 [==============================] - 96s 672us/step - loss: 4.1427 - acc: 0.1966
Epoch 78/150
14326

143265/143265 [==============================] - 93s 648us/step - loss: 2.7243 - acc: 0.3897
Epoch 138/150
143265/143265 [==============================] - 92s 640us/step - loss: 2.7002 - acc: 0.3939
Epoch 139/150
143265/143265 [==============================] - 91s 632us/step - loss: 2.6704 - acc: 0.4000
Epoch 140/150
143265/143265 [==============================] - 90s 631us/step - loss: 2.6437 - acc: 0.4036

Epoch 00140: saving model to models/weights.140-2.64-bigger.hdf5
Epoch 141/150
143265/143265 [==============================] - 91s 632us/step - loss: 2.6202 - acc: 0.4088
Epoch 142/150
143265/143265 [==============================] - 91s 632us/step - loss: 2.5916 - acc: 0.4128
Epoch 143/150
143265/143265 [==============================] - 91s 632us/step - loss: 2.5623 - acc: 0.4173
Epoch 144/150
143265/143265 [==============================] - 90s 631us/step - loss: 2.5390 - acc: 0.4219
Epoch 145/150
143265/143265 [==============================] - 91s 633us/step - loss: 2.5134